# Cleaning and Analyzing Employee Exit Surveys

Feedback from employee exit surveys can provide powerful insight into a company’s culture. It doesn't matter how excellent a company is, people are eventually going to leave. Exit surveys allows leaving employees to share their unique opinions. This can help companies in mitigating the many costs of losing other employees in the future.

![](https://images.squarespace-cdn.com/content/v1/5dd58dee80e8fe0621b369a3/1630337526747-0Y46T61FP1AFLQ9X9A77/Employee+Exiting+An+Organization?format=1000w)

Image source: [Skywalk Group](https://skywalkgroup.com/blog/post/the-importance-of-exit-interviews)

## Project and Data Overview
In this Project, we'll work with exit surveys from employees of the Department of Education, Training and Employment (DETE) and the Technical and Further Education (TAFE) institute in Queensland, Australia. 

The DETE exit survey data can be found [here](https://data.gov.au/dataset/ds-qld-fe96ff30-d157-4a81-851d-215f2a0fe26d/details?q=exit%20survey). However, the original TAFE exit survey data is no longer available. Some modifications have been made to the original datasets to make them easier to work with, especially changing the encoding from cp1252 to UTF-8.

## Business Problem
We will play the role of data analyst and pretend our stakeholders want to know the following:
- Is some dissatisfaction causing newer and older employees to resign from the institute?
- If a dissatisfaction is present, how does vary within the different age groups at the instititute? 
- Are females more likely to resign than males?

The stakeholders want us to combine the results from both surveys and answer these questions. Although both surveys used the same template, one of them had customized answers. 

## Data Dictionary
A data dictionary wasn't provided with the dataset. In a job setting, we'd **make sure** to meet with a manager to confirm the definitions of the data. For this project, we'll use our general knowledge to define the columns.

From *`dete_survey.csv`*, we will focus on the following columns:
- `ID`: An id used to identify the participant of the survey.

- `SeparationType`: The reason why the person's employment ended.

- `Cease Date`: The year or month the person's employment ended.

- `DETE Start Date`: The year the person began employment with the DETE.

From *`tafe_survey.csv`*, we will focus on the following columns:
- `Record ID`: An id used to identify the participant of the survey.

- `Reason for ceasing employment`: The reason why the person's employment ended.

- `LengthofServiceOverall`. Overall Length of Service at Institute (in years): The length of the person's employment (in years).
